# Liquid Formulations Stability Classifier
## Aniket Chitre, University of Cambridge

This notebook is used to train a phase stability classifier over an experimental formulations dataset to guide which samples to prepare in the next DoE (design of experiments) step. The notebook reads several CSV files generated via a sister DoE code in R:

* `ExperimentalSet.csv` the pre-processed experimental dataset where the formulations are represented by the scaled  concentrations of the surfactants' functional groups and the polymer & thickener concentrations.
* `CandidateSet.csv` the full candidate list of experiments in the same pre-processed format as the experimental data.
* `CandidateDesign.csv` the full candidate list of experiments in the format of the MaxProQQ (DoE package) design variables.

The code trains and tunes several machine learning models looking for the best phase stability classifier for the given formulations problem. This is then used to predict the stability of the candidate points and restrict the full candidate set to a more suitable sub-set of potential experiments. These are exported in the `RestrictedCandDesign.csv` file which is read in the DoE programme written in R, to ultimately generate the next suggested set of experiments.

# 0. Import Packages

In [ ]:
import time
import pandas as pd
import numpy as np
import matplotlib
%matplotlib inline
matplotlib.rcParams["figure.dpi"] = 600
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
pio.renderers.default='notebook'
from IPython.display import Image
import re 
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from bayes_opt import BayesianOptimization
import xgboost
from xgboost import XGBClassifier
import pickle
import shap

# 1. Load Experimental Dataset

In [ ]:
df = pd.read_csv('ExperimentalSet/Expt_P1_T1.csv', index_col=[0])
df

In [ ]:
# x_P1T1 = pd.read_csv('ExperimentalSet/Expt_P1_T1.csv', index_col=[0])
# x_P1T2 = pd.read_csv('ExperimentalSet/Expt_P1_T2.csv', index_col=[0])
# x_P4T1 = pd.read_csv('ExperimentalSet/Expt_P4_T1.csv', index_col=[0])

# X_P1T1 = x_P1T1.iloc[:, :-1]
# X_P1T2 = x_P1T2.iloc[:, :-1]
# X_P4T1 = x_P4T1.iloc[:, :-1]

# y_P1T1 = x_P1T1.iloc[:, -1]
# y_P1T2 = x_P1T2.iloc[:, -1]
# y_P4T1 = x_P4T1.iloc[:, -1]

In [ ]:
# split the dataframe into input variables (features) & output
# last column of the df is the classification target = phase stability
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

# 2. Machine Learning

In [ ]:
# Stratified train: test split due to the data imbalance: majority unstable formulations
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=42)

# X_train_P1T1, X_test_P1T1, y_train_P1T1, y_test_P1T1 = train_test_split(X_P1T1, y_P1T1, stratify=y_P1T1, test_size=0.25, random_state=42)
# X_train_P1T2, X_test_P1T2, y_train_P1T2, y_test_P1T2 = train_test_split(X_P1T2, y_P1T2, stratify=y_P1T2, test_size=0.25, random_state=42)
# X_train_P4T1, X_test_P4T1, y_train_P4T1, y_test_P4T1 = train_test_split(X_P4T1, y_P4T1, stratify=y_P4T1, test_size=0.25, random_state=42)

In [ ]:
print('Training set:')
print(y_train.value_counts())
print('\n')
print('Test set:')
print(y_test.value_counts())

*Key: True = stable, False = unstable*

## 2.1 Shortlisting Models

### Setting a Baseline

Before we delve into tuning more complex models, it is often useful to train some common model types with their default hyperparameters to set a baseline with which to compare the more carefully selected and tuned models.

We will begin looking at logistic regression, Naive Bayes and decision tree classifiers.

In [ ]:
# call the models 
logreg_clf = LogisticRegression()
nb_model = GaussianNB()
dtc_model = DecisionTreeClassifier(random_state=42)

#fit the models 
logreg_clf.fit(X_train, y_train)
nb_model.fit(X_train, y_train)
dtc_model.fit(X_train, y_train)

# make predictions on the test ste
logreg_pred = logreg_clf.predict(X_test)
nb_pred = nb_model.predict(X_test)
dtc_pred = dtc_model.predict(X_test)

# confusion_matrix(y_true, y_pred) - it's essential to get the order of the arguments correct! 

print('Logistic Regression')
print(confusion_matrix(y_test, logreg_pred))

print('\n')

print('Naive Bayes')
print(confusion_matrix(y_test, nb_pred))

print('\n')

print('Decision Tree')
print(confusion_matrix(y_test, dtc_pred))

The quickest way for me to assess the performance of the models is to look at the results on the test set presented in the form of a confusion matrix.

In [ ]:
Image(filename='ImagesforJupyter/ConfusionMatrix.png') 

The above models are only used to provide a baseline, as they have some known shortcomings and there are likely to be better models available:

* Logistic regression is a type of generalised linear model, however, the formulation's design space is non-linear and highly complex.
* Similarly, there are issues with the Naive Bayes (NB) model, as there are lots of zero counts for functional groups. It's known that the NB model struggles with this (e.g., https://www.atoti.io/articles/how-to-solve-the-zero-frequency-problem-in-naive-bayes)
* And decision trees typically overfit as the structure of a tree can be highly dependent on the training data. 

The most promising models to test and tune are random forests and gradient boosted algorithms, which are both ensembles of decision trees and typically perform very well on structured tabular data, like this liquid formulations problem. Additionally, support vector machines are another method to test more rigourously. More expensive ML methods, like neural networks, are being avoided, as there is not enough data relative to the number of paramteres to train such an architecture. 

In [ ]:
rf_clf = RandomForestClassifier(random_state=42) # random forest with its default parameters

svc = SVC(random_state=42) # Support Vector machine with its default parameters 

gbc = GradientBoostingClassifier(random_state=42) # Gradient Boosting classifier with its default parameters

rf_clf.fit(X_train, y_train)

svc.fit(X_train, y_train)

gbc.fit(X_train, y_train)

rf_pred  = rf_clf.predict(X_test)

svc_pred = svc.predict(X_test)

gbc_pred = svc.predict(X_test)

*N.b.* See this article (https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-018-2264-5) which suggests random forests are the new "default choice" taking over from logistic regression. This is also commonly documented in the online literature from several machine learning blogs.



In [ ]:
print('Results presented as confusion matrix')
print('\n')

print('Random Forest')
print(confusion_matrix(y_test, rf_pred))

print('\n')

print('Support Vector Machine')
print(confusion_matrix(y_test, svc_pred,))

print('\n')

print('Gradient Boosting Classifier')
print(confusion_matrix(y_test, gbc_pred))

In [ ]:
print("ROC AUC Score: {}".format(round(roc_auc_score(y_test, gbc_pred),2)))

print("F1 Score: {}".format(round(f1_score(y_test, gbc_pred, average='weighted'),2)))

## 2.2 Tuning Candidate Models

There are three main models which I am interested in training and tuning and then I will select the best performing one to develop the restricted candidate set of experiments for this model-based DoE.

1. Random Forests 
2. Support Vector Machines
3. Xgboost - Gradient Boosting

Models #1 and 3 have a particularly nice feature that we can intriniscally provide an estimate for feature importance and the surfactants in the formulation have purposefully been represented as concentrations of their functional groups to give the results a level of chemical interpretability.

We will test out three different methods of hyperparameter tuning:

1. Use of **GridSearchCV** to try out all possible, user-defined, combinations of hyperparameters.
2. Use of **RandomSearchCV** to try out possible permutations of hyperparameters sampled from a defined search sapce.
3. **Bayesian optimisation** (BO) to search for the optimal hyperparameters guided through an informed learning approach.

The `bayes_opt` package (https://github.com/fmfn/BayesianOptimization) is used for the BO and several tutorials can be found online how to apply BO for hyperparmeter tuning: https://www.analyticsvidhya.com/blog/2021/05/bayesian-optimization-bayes_opt-or-hyperopt/

In both cases the **weighted F1 score** will be used as the evaluation metric. Particularly for an imbalanced dataset like this it's important not to look at accuracy.

The F1 score is defined as the harmonic mean between precision and recall 


$$ F_1 = \frac{2 \cdot precision \cdot recall}{(precision + recall)} $$

where $precision = \frac{TP}{TP + FP} $ and $ recall = \frac{TP}{TP + FN}$

Recall is also known as sensitivity or the true positive rate. And if you take the F1 scores for the TP and TN clases and weight them by the proportion in each of the False and True classes, then we get the weighted F1 score. This is a suitable evaluation metric for this dataset. **The best model will be selected as the model with the highest weighted F1 score when evaluated on the held-out test set.**

When training the models, **k-folds cross validation with k=6 will be used**. An illustration is provided below (where k = 5).

In [ ]:
Image(filename='ImagesforJupyter/kFoldsCV.png') 

In [ ]:
cv_n = 6

### 2.2.1 Random Forest

#### a) GridSearchCV

In [ ]:
start = time.time()

# Model documentation: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

rf_params = dict(
    n_estimators = [5, 10, 15, 20, 50, 75, 100], # number of trees in the forest 
    criterion = ["gini"], # default criterion & the more efficient metric vs. entropy
    max_depth = [None], # default = None, exapnds nodes until all leaves are pure or until all leaves contain less than min_samples_split
    min_samples_split = [2, 3, 4, 5], # the minimum number of samples required to split an internal node 
    min_samples_leaf = [1, 2, 3, 4], # the minimum number of samples to be at a leaf node - this & above help prevent overfitting - trees don't have to go down to always producing pure nodes 
    max_features = [0.25, 0.5, 0.75, None], # the number of features to consider when looking at the best split - not the total number features the model considers! 
    class_weight=[{False: 1, True: 1},
                  {False: 1, True: 2}] # investigate the effect of biasing the model to the true class    
)

rf = RandomForestClassifier(random_state=42, n_jobs=-1) # n_jobs = -1 instructs the computer to use all available CPU power training the model 

rf_gcv = GridSearchCV(estimator=rf, param_grid=rf_params, cv=cv_n, n_jobs=-1, verbose=1, scoring='f1_weighted')

rf_gcv.fit(X_train, y_train)

print('It takes %s seconds' % round((time.time() - start)))

In [ ]:
print(f"Best model accuracy: {round(rf_gcv.best_score_,3)}")

print(f"Best hyperparamters: {rf_gcv.best_params_}")


rf_GridSearch_best_model = rf_gcv.best_estimator_

#### Evaluate Results 

Use the performance on the train set vs. test set to gauge if the model is overfitting.

In [ ]:
y_train_rf_BestGridSearch = rf_GridSearch_best_model.predict(X_train)

In [ ]:
print(classification_report(y_train, y_train_rf_BestGridSearch))

In [ ]:
print(confusion_matrix(y_train, y_train_rf_BestGridSearch))

In [ ]:
y_test_rf_BestGridSearch = rf_GridSearch_best_model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_test_rf_BestGridSearch))

In [ ]:
print("ROC AUC Score: {}".format(round(roc_auc_score(y_test, y_test_rf_BestGridSearch),2)))

print("F1 Score: {}".format(round(f1_score(y_test, y_test_rf_BestGridSearch, average='weighted'),2)))

In [ ]:
print(confusion_matrix(y_test, y_test_rf_BestGridSearch))

#### b) RandomSearchCV

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 200, num = 20)]
# Number of features to consider at every split
max_features = [x for x in np.linspace(start=0.2, stop=1.0, num=10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 6 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 400, cv=cv_n, verbose=1, random_state=42, n_jobs = -1, scoring='f1_weighted')
# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
print(f"Best model accuracy: {round(rf_random.best_score_,3)}")

print(f"Best hyperparamters: {rf_random.best_params_}")


rf_RandomSearch_best_model = rf_random.best_estimator_

In [ ]:
y_train_rf_BestRandomSearch = rf_RandomSearch_best_model.predict(X_train)
y_test_rf_BestRandomSearch = rf_RandomSearch_best_model.predict(X_test)

In [ ]:
print("ROC AUC Score: {}".format(round(roc_auc_score(y_test, y_test_rf_BestRandomSearch),2)))

print("F1 Score: {}".format(round(f1_score(y_test, y_test_rf_BestRandomSearch, average='weighted'),2)))

In [ ]:
print(confusion_matrix(y_train, y_train_rf_BestRandomSearch))

In [ ]:
print(confusion_matrix(y_test, y_test_rf_BestRandomSearch))

#### c) Bayesian Optimisation

In [ ]:
def rf_clf_bo(n_estimators, min_samples_split, min_samples_leaf, max_features):
    
    params_rf = {}
    params_rf['n_estimators'] = round(n_estimators) # round so only tries integer values of n_estimators 
    params_rf['min_samples_split'] = round(min_samples_split)
    params_rf['min_samples_leaf'] = round(min_samples_leaf)
    params_rf['max_features'] = max_features
    
    scores = cross_val_score(RandomForestClassifier(random_state=42, n_jobs=-1, **params_rf),
                             X_train, y_train, scoring='f1_weighted', cv=cv_n).mean() # tried lots of different score metrics, yet GridSeach CV result above outperforms BO performance on test class 
    score = scores.mean()
    return score

In [ ]:
start = time.time()

params_rf = {
    'n_estimators': (5, 100),
    'min_samples_split': (2,10),
    'min_samples_leaf': (2,10),
    'max_features': (0.25,1.0)
}

rf_bo = BayesianOptimization(rf_clf_bo, params_rf, random_state=42)
rf_bo.maximize(init_points=400, n_iter=100) # starts with 100 random points & goes for 400 iterations 
print('It takes %s seconds' % round((time.time() - start)))

In [ ]:
params_BO_rf = rf_bo.max['params']
params_BO_rf['min_samples_leaf'] = round(params_BO_rf['min_samples_leaf']) 
params_BO_rf['min_samples_split'] = round(params_BO_rf['min_samples_split'])
params_BO_rf['n_estimators'] = round(params_BO_rf['n_estimators'])

In [ ]:
rf_BO_best_model = RandomForestClassifier(**params_BO_rf, random_state=42)
rf_BO_best_model.fit(X_train, y_train)

In [ ]:
y_train_rf_BestBO = rf_BO_best_model.predict(X_train)

In [ ]:
print(classification_report(y_train, y_train_rf_BestBO))

In [ ]:
print(confusion_matrix(y_train, y_train_rf_BestBO))

In [ ]:
y_test_rf_BestBO = rf_BO_best_model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_test_rf_BestBO))

In [ ]:
print("ROC AUC Score: {}".format(round(roc_auc_score(y_test, y_test_rf_BestBO),2)))

print("F1 Score: {}".format(round(f1_score(y_test, y_test_rf_BestBO, average='weighted'),2)))

In [ ]:
print(confusion_matrix(y_test, y_test_rf_BestBO))

### Model Interpretability

One of the advantages of tree-based models is that they have an in-built method for looking at the feature importances. Similarly, the SHAP library (https://github.com/slundberg/shap) can also be used to look at the contribution of features to the final predictions.

Before looking into these methods it is first essential to understand that **feature importances/SHAP values can only be relied on if the model itself is accurate (low bias, low variance)**.

There is also a key difference between feature importances and SHAP values. This post provides a good explanation: (https://datascience.stackexchange.com/questions/99650/difference-between-feature-effect-and-feature-importance) 

*"The goal of SHAP is to explain the prediction of an instance x by computing the contribution of each feature to the prediction. [...]" SHAP feature importance is an alternative to permutation feature importance. There is a big difference between both importance measures: permutation feature importance is based on the decrease in model performance. SHAP is based on the magnitude of feature attributions.*

This is a nice blog post for different ways to compute feature importance with RFs: https://mljar.com/blog/feature-importance-in-random-forest/

See the following tutorial on **SHAP feature explanations**: https://www.youtube.com/watch?v=9haIOplEIGM&ab_channel=DeepFindr

#### Permutation Feature Importance

In [ ]:
rf_features_sorted_idx = best_model.feature_importances_.argsort()
plt.barh(X.columns[rf_features_sorted_idx], best_model.feature_importances_[rf_features_sorted_idx])
plt.xlabel("Random Forest Feature Importance", fontsize=14);
plt.xticks(fontsize=14);
plt.yticks(fontsize=14);
plt.show()

#### SHAP Feature Explanations

In [ ]:
rf_explainer = shap.TreeExplainer(best_model) 
rf_shap_values = rf_explainer(X_test)

With the random forest model, we will get SHAP values for all of the classes, in this binary classification, for both the unstable (False) and stable (True) samples, hence below we want to look at: 

`rf_shap_values[instance,:,class]` where `instance` refers to the sample and `class=1` will be the stable class and we will want to look at `:` all the different features.

In [ ]:
shap.initjs()
shap.plots.force(rf_shap_values[4,:,1]) # binary output - 0 = False class, 1 = True class 

Here we are just looking at samples one-by-one, a full summary of feature importances can also be visualised with the `beeswarm()` plot, however, this is only looked at for the best selected model below in Section 2.3.

What we see from this plot is:

* `base value` is the `mean(model.predict(X))` - given we didn't know any of the features for the particular sample, we would guess the probability of the output to be the mean of the full set - overall probability of a stable sample.
* Then we look at how the features of the particular sample actually drive the prediction to `f(x)` with the red features increasing the likelihood of stability in this scenario & blue decreasing the likelihood.

### 2.2.2 Support Vector Machine

In [ ]:
start = time.time()

# Model documentation: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

svc_params = dict(
    C = [2, 4, 8, 10, 20], # regularisation parameter - strength of regularisation is inversely proportional to C 
    kernel = ["linear", "poly", "rbf", "sigmoid"], 
    degree = [2, 3, 4, 5], # degree of polynomal, ignored if other kernels selected
    class_weight=[{False: 1, True: 1},
                  {False: 1, True: 2}] # investigate the effect of biasing the model to the true class    
)


svc = SVC(random_state=42, probability=True)

svc_gcv = GridSearchCV(estimator=svc, param_grid=svc_params, cv=cv_n, n_jobs=-1, verbose=1, scoring='f1_weighted')

svc_gcv.fit(X_train, y_train)

print('It takes %s seconds' % round((time.time() - start)))

In [ ]:
print(f"Best model accuracy: {round(svc_gcv.best_score_,3)}")

print(f"Best hyperparamters: {svc_gcv.best_params_}")


svc_GridSearch_best_model = svc_gcv.best_estimator_

#### Evaluate Results 

In [ ]:
y_train_svc_BestGridSearch = svc_GridSearch_best_model.predict(X_train)

In [ ]:
print(classification_report(y_train, y_train_svc_BestGridSearch))

In [ ]:
print(confusion_matrix(y_train, y_train_svc_BestGridSearch))

In [ ]:
y_test_svc_BestGridSearch = svc_GridSearch_best_model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_test_svc_BestGridSearch))

In [ ]:
print("ROC AUC Score: {}".format(round(roc_auc_score(y_test, y_test_svc_BestGridSearch),2)))

print("F1 Score: {}".format(round(f1_score(y_test, y_test_svc_BestGridSearch, average='weighted'),2)))

In [ ]:
print(confusion_matrix(y_test, y_test_svc_BestGridSearch))

### 2.2.3 XGBoost

With XGBoost the square brackets which are in the SMILES fragments cause a problem with the feature names as this algorithm cannot accept this input. https://stackoverflow.com/questions/48645846/pythons-xgoost-valueerrorfeature-names-may-not-contain-or

Therefore, regex operations are used in the following code block to substitute the `[` or `]` for `_`

In [ ]:
regex = re.compile(r"\[|\]|<", re.IGNORECASE)


# copy the feature sets as don't want to overwrite these directly
X_train_xgb = X_train.copy()
X_test_xgb  = X_test.copy()

X_train_xgb.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']'))) else col for col in X_train_xgb.columns.values]
X_test_xgb.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']'))) else col for col in X_test_xgb.columns.values]

#### a) GridSearchCV

The following two blog posts provide a great explanation of the hyperparameters available to be tuned with XGBoost:

1. https://neptune.ai/blog/xgboost-vs-lightgbm
2. https://www.kaggle.com/code/prashant111/a-guide-on-xgboost-hyperparameters-tuning/notebook

Josh Starmer's StatQuest provides a great explanation of how XGBoost works for classification: https://www.youtube.com/watch?v=8b1JEDvenQU

In [ ]:
start = time.time()

xgb_params = dict(
    learning_rate = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5], # lower learning rate makes the model more conservative - also increases training time 
    n_estimators = [10, 20, 50, 100, 150], # number of weak learners - decision trees 
    max_depth = [3, 4, 5, 6, 7],  # maximum depth of a tree - lower value will result in less overfitting
    min_child_weight = [2, 4, 8], # higher values used to control overfitting
    gamma = [0, 0.1, 0.5, 1, 10], # gamma specifies the minimum loss reduction required to make a split
    colsample_bytree = [0.5, 0.75, 0.95] # samples a fraction of the columns when constructing each tree - again, to prevent overfitting
)

xgb = XGBClassifier(booster='gbtree', objective='binary:logistic', random_state=42, n_jobs=-1) # n_jobs = -1 instructs the computer to use all available CPU power training the model 

xgb_gcv = GridSearchCV(estimator=xgb, param_grid=xgb_params, cv=cv_n, n_jobs=-1, verbose=1, scoring='f1_weighted')

xgb_gcv.fit(X_train_xgb, y_train)

print('It takes %s seconds' % round((time.time() - start)))

In [ ]:
print(f"Best model accuracy: {round(xgb_gcv.best_score_,3)}")

print(f"Best hyperparamters: {xgb_gcv.best_params_}")


xgb_GridSearch_best_model = xgb_gcv.best_estimator_

In [ ]:
y_train_xgb_BestGridSearch = xgb_GridSearch_best_model.predict(X_train_xgb)

In [ ]:
print(classification_report(y_train, y_train_xgb_BestGridSearch))

In [ ]:
print(confusion_matrix(y_train, y_train_xgb_BestGridSearch))

In [ ]:
y_test_xgb_BestGridSearch = xgb_GridSearch_best_model.predict(X_test_xgb)

In [ ]:
print(classification_report(y_test, y_test_xgb_BestGridSearch))

In [ ]:
print("ROC AUC Score: {}".format(round(roc_auc_score(y_test, y_test_xgb_BestGridSearch),2)))

print("F1 Score: {}".format(round(f1_score(y_test, y_test_xgb_BestGridSearch , average='weighted'),2)))

In [ ]:
print(confusion_matrix(y_test, y_test_xgb_BestGridSearch))

#### b) RandomSearchCV 

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 200, num = 20)]
# Learning rate - lower learning rate makes model more conservative
learning_rate = [x for x in np.linspace(start=0.05, stop=0.5, num=10)]
# Maximum depth of a tree - lower value will result in less overfitting
max_depth = [int(x) for x in np.linspace(start=3, stop=10, num=5)]
# Min child weight - higher values used to control overfitting
min_child_weight = [2, 4, 8]
# Gamma specifies the minimum loss reduction required to make a split 
gamma = [x for x in np.logspace(start=-3, stop=3, num = 6)]
# Colsample_bytree 
colsample_bytree = [x for x in np.linspace(start=0.4, stop=1.0, num=8)]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'learning_rate': learning_rate,
               'max_depth': max_depth,
               'min_child_weight': min_child_weight,
               'gamma': gamma,
               'colsample_bytree': colsample_bytree}

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
xgb = XGBClassifier(booster='gbtree', objective='binary:logistic', random_state=42, n_jobs=-1)

# Random search of parameters, using 6 fold cross validation, 
# search across 100 different combinations, and use all available cores
xgb_random = RandomizedSearchCV(estimator = xgb, param_distributions = random_grid, n_iter = 800, cv=cv_n, verbose=1, random_state=42, n_jobs = -1, scoring='f1_weighted')
# Fit the random search model
xgb_random.fit(X_train_xgb, y_train)

In [ ]:
xgb_random.best_params_

In [ ]:
y_test_xgb_BestRandomSearch = xgb_random.predict(X_test_xgb)

In [ ]:
print("ROC AUC Score: {}".format(round(roc_auc_score(y_test, y_test_xgb_BestRandomSearch),2)))

print("F1 Score: {}".format(round(f1_score(y_test, y_test_xgb_BestRandomSearch, average='weighted'),2)))

#### c) Bayesian Optimisation

In [ ]:
def xgb_clf_bo(n_estimators, max_depth, min_child_weight, learning_rate, gamma, colsample_bytree):
        
    params_xgb = {}
    params_xgb['n_estimators'] = round(n_estimators) # round so only tries integer values of n_estimators 
    params_xgb['max_depth'] = round(max_depth)
    params_xgb['min_child_weight'] = round(min_child_weight)
    params_xgb['learning_rate'] = learning_rate
    params_xgb['gamma'] = round(gamma)
    params_xgb['colsample_bytree'] = colsample_bytree
        
    scores = cross_val_score(XGBClassifier(booster='gbtree', objective='binary:logistic', random_state=42, n_jobs=-1, **params_xgb),
                             X_train_xgb, y_train, scoring='f1_weighted', cv=cv_n).mean() # tried lots of different score metrics, yet GridSeach CV result above outperforms BO performance on test class 
    score = scores.mean()
    return score

In [ ]:
start = time.time()

params_xgb = {
    'n_estimators': (10, 200),
    'max_depth': (3,7),
    'min_child_weight': (2,8),
    'learning_rate': (0.05, 0.5),
    'gamma': (0.1, 1000),
    'colsample_bytree': (0.5, 0.9)
}

xgb_bo = BayesianOptimization(xgb_clf_bo, params_xgb , random_state=42)
xgb_bo.maximize(init_points=400, n_iter=100) # starts with 100 random points & goes for 400 iterations 
print('It takes %s seconds' % round((time.time() - start)))

In [ ]:
params_BO_xgb = xgb_bo.max['params']
params_BO_xgb['n_estimators'] = round(params_BO_xgb['n_estimators']) 
params_BO_xgb['max_depth'] = round(params_BO_xgb['max_depth']) 
params_BO_xgb['min_child_weight'] = round(params_BO_xgb['min_child_weight'])

In [ ]:
xgb_BO_best_model = XGBClassifier(booster='gbtree', objective='binary:logistic', random_state=42, n_jobs=-1, **params_BO_xgb)
xgb_BO_best_model.fit(X_train_xgb, y_train)

In [ ]:
y_train_xgb_BestBO = xgb_BO_best_model.predict(X_train_xgb)

In [ ]:
print(classification_report(y_train, y_train_xgb_BestBO))

In [ ]:
print(confusion_matrix(y_train, y_train_xgb_BestBO))

In [ ]:
y_test_xgb_BestBO = xgb_BO_best_model.predict(X_test_xgb)

In [ ]:
print(classification_report(y_test, y_test_xgb_BestBO))

In [ ]:
print("ROC AUC Score: {}".format(round(roc_auc_score(y_test, y_test_xgb_BestBO),2)))

print("F1 Score: {}".format(round(f1_score(y_test, y_test_xgb_BestBO, average='weighted'),2)))

In [ ]:
print(confusion_matrix(y_test, y_test_xgb_BestBO))

### Model Interpretability

#### Permutation Feature Importance

In [ ]:
xgb_features_sorted_idx = xgb_BO_best_model.feature_importances_.argsort()
plt.barh(X.columns[xgb_features_sorted_idx], xgb_BO_best_model.feature_importances_[xgb_features_sorted_idx])
plt.xlabel("XGBoost Feature Importance");

#### SHAP 

In [ ]:
xgb_explainer = shap.TreeExplainer(model=xgb_GridSearch_best_model, data=X_train_xgb, model_output='probability')
xgb_shap_values = xgb_explainer(X_test_xgb);

In [ ]:
shap.initjs()
shap.plots.force(xgb_shap_values[7])

## 2.3 Select best model

In [ ]:
best_model = rf_RandomSearch_best_model

In [ ]:
#best_model.best_params_
best_model.get_params

In [ ]:
filename = "PastModels/P4_T1_Full.pickle"

# save model
pickle.dump(best_model, open(filename, "wb"))

In [ ]:
filename = "PastModels/P1_T1_Full.pickle"

# load model
loaded_model = pickle.load(open(filename, "rb"))
best_model = loaded_model

In [ ]:
# P1T1_model = pickle.load(open("PastModels/P1_T1_Full.pickle", "rb"))
# P1T2_model = pickle.load(open("PastModels/P1_T2_Full.pickle", "rb"))
# P4T1_model = pickle.load(open("PastModels/P4_T1_Full.pickle", "rb"))

In [ ]:
Image(filename='ImagesforJupyter/ROC.png')

It is useful to plot the Reciever Operating Characteristic (ROC) curve to visualise the performance of the classifier and area under the curve, ROC AUC, has been a metric used throughout this notebook. A ROC AUC = 1 would be a perfect classifier, whereas 0.5 would be a random classifier. 

https://deepchecks.com/question/what-is-a-good-roc-curve-score/ provides some typical guidelines for a good ROC AUC score.

Remember the true positive rate is equivalent to recall. So as we increase the false positive rate, i.e., allow more negative examples to be incorrectly classified as true, we are bound to also capture, recall, more of the true samples in general. Therefore, at the extremas where you accept no false positives, you also can't capture any true positives, however, at the other end, where you allow everything in, you will also have a true positive rate = 1.

In [ ]:
if type(best_model) != xgboost.sklearn.XGBClassifier:
    y_pred_proba = best_model.predict_proba(X_test)[::,1]
    fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
    auc = roc_auc_score(y_test, y_pred_proba)
    f1_weighted = f1_score(y_test, best_model.predict(X_test), average='weighted')
else:
    y_pred_proba = best_model.predict_proba(X_test_xgb)[::,1]
    fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
    auc = roc_auc_score(y_test, y_pred_proba)
    f1_weighted = f1_score(y_test, best_model.predict(X_test_xgb), average='weighted')


#create ROC curve
plt.plot(fpr,tpr,label="ROC AUC="+str(round(auc,2))+"\nF1_weighted="+str(round(f1_weighted,2)))
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc=4)
plt.title('(P4, T1)')
plt.show()

In [ ]:
# models = [P1T1_model, P1T2_model, P4T1_model]
# X_data = [X_test_P1T1, X_test_P1T2, X_test_P4T1]
# y_data = [y_test_P1T1, y_test_P1T2, y_test_P4T1]
# sub_systems = ['(P1, T1)', '(P1, T2)', '(P4, T1)']

# fpr_res, tpr_res, auc_res, f1_weighted_res = [], [], [], []

# for i in range(len(models)):
#     if type(models[i]) != xgboost.sklearn.XGBClassifier:
#         y_pred_proba = models[i].predict_proba(X_data[i])[::,1]
#         fpr, tpr, _ = roc_curve(y_data[i],  y_pred_proba)
#         auc = roc_auc_score(y_data[i], y_pred_proba)
#         f1_weighted = f1_score(y_data[i], models[i].predict(X_data[i]), average='weighted')
#     else:
#         print("Used XGBoost model")
#         # y_pred_proba = models[i].predict_proba(X_test_xgb)[::,1]
#         # fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
#         # auc = roc_auc_score(y_test, y_pred_proba)
#         # f1_weighted = f1_score(y_test, best_model.predict(X_test_xgb), average='weighted')
#     fpr_res.append(fpr)
#     tpr_res.append(tpr)
#     auc_res.append(auc)
#     f1_weighted_res.append(f1_weighted)


# #create ROC curve
# for i in range(len(models)):
#     plt.plot(fpr_res[i], tpr_res[i], label=sub_systems[i]+"\nROC AUC="+str(round(auc_res[i],2))+"\n$F1_{weighted}$="+str(round(f1_weighted_res[i],2)))
#     plt.ylabel('True Positive Rate')
#     plt.xlabel('False Positive Rate')
#     plt.legend(loc=4)
# plt.show()

These metrics, the ROC AUC and weighted F1 score, are the ones recorded offline, to track, ideally model improvement, across time as more data is generated in the project. 

One notices the ROC AUC score in the figure's legend is higher than the ROC AUC score computed for the same model. The following post provides a good explanation: https://stats.stackexchange.com/questions/521503/what-is-the-reason-for-the-difference-in-auc-from-probabilities-vs-auc-from-final

Essentially, here we computed the true positive and false positive rates looking at the predicted probabilities for the outputs, not just the class classifications. For e.g., if we compute the probability as 0.6 to be stable and in fact the sample is unstable, i.e., a false positive, this would be penalised more heavily in the binary classification using the `model.predict()` method vs. the `model.predict_proba()` method as utilised here. In this case we are also assigning a 0.4 probability that the model could in fact be unstable, so this point would be penalised less harshly in computing the overall ROC AUC metric. Hence, we can rationalise why the ROC AUC score in this sub-section, 2.3, is higher than that obtain in 2.2 for the same model.

#### Model Interpretability and Feature Importance

In [ ]:
if type(best_model) != xgboost.sklearn.XGBClassifier:
    explainer = shap.TreeExplainer(model=best_model);
    shap_values = explainer(X_test);
    shap.plots.beeswarm(shap_values[:,:,1], max_display=10);
else:
    explainer = shap.TreeExplainer(model=best_model, data=X_train_xgb, model_output='probability', show=False);
    shap_values = explainer(X_test);
    shap.plots.beeswarm(shap_values, show=False);

# 3. Applying the Best Model to the Candidate Set

*N.b.* By default, I have commented out this section so I can restart the kernel & run all without interfering/re-generating important files in the DoE directory.

In [ ]:
# cand_df = pd.read_csv('CandidateSet.csv', index_col=[0])
# cand_df

In [ ]:
# since xgboost doesn't accept the `[`, `]` in the SMILES fragments, regex operations were used to change the column names
# the xgboost model trained on X_train_xgb, X_test_xgb with modified column names
# thus if the best_model is xgboost, the same regex modification needs to be applied to the cand_df

# if type(best_model) != xgboost.sklearn.XGBClassifier:
#     cand_prob = best_model.predict_proba(cand_df)
# else:
#     cand_df.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in cand_df.columns.values]
#     cand_prob = best_model.predict_proba(cand_df)
    
# cand_stability_df = pd.DataFrame(cand_prob, columns=['Unstable', 'Stable'])
# cand_stability_df.index += 1 # R != 0 indexing like Python
# cand_stability_df

In [ ]:
# # this snippet is most likely not needed anymore - it looks at the tail of candidates cut-off by a top % stable criterion

# top_stable_portion = 0.1 
# sorted_cand = cand_stability_df.sort_values('Stable', ascending=False)
# sorted_cand[:int(len(sorted_cand)*top_stable_portion)].tail()

In [ ]:
# stable_prob_cutoff = 0.6
# cand_stability_df[cand_stability_df['Stable'] > stable_prob_cutoff]

In [ ]:
# stable_idx = cand_stability_df[cand_stability_df['Stable'] > stable_prob_cutoff].index

In [ ]:
# cand_design = pd.read_csv('CandDesign.csv', index_col=[0]) # this candidate design was converted to the candidate set in R
# # we can filter on the candidate design from the most stable samples predicted from the candidate set - it's equivalent.
# restricted_cand = cand_design.filter(items = stable_idx, axis=0)
# restricted_cand

In [ ]:
# top_x = round(len(restricted_cand)*100/len(cand_df),2)
# print(f"The top {top_x}% of stable candidates have been selected from the candidate set.")
# print(f"This is for a minimum probability of stability of {stable_prob_cutoff}.")

In [ ]:
# restricted_cand.to_csv('RestrictedCandDesign.csv')